In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

def save_npz_images(npz_path, save_dir, num_images=50):
    """
    从 .npz 文件中加载 RGB 图片，并保存到指定目录。

    参数:
    - npz_path (str): .npz 文件的路径
    - save_dir (str): 图片保存的目标目录
    - num_images (int): 需要保存的图片数量（默认为 50）
    """
    # 确保保存目录存在
    os.makedirs(save_dir, exist_ok=True)

    # 加载 .npz 文件
    data = np.load(npz_path)

    # 获取图片数组，通常存储在 'arr_0'，可能需要检查具体 key
    images = data["arr_0"]  # shape: (N, H, W, C) 其中 C 可能是 3(RGB) 或 4(RGBA)

    # 确保不超过总图片数
    num_images = min(num_images, images.shape[0])

    # 遍历并保存图片
    for i in range(num_images):
        img = images[i]

        # 确保图像是 uint8 类型
        if img.dtype != np.uint8:
            img = (img * 255).astype(np.uint8)  # 归一化到 [0,255]

        # 如果图像是 RGBA（C=4），转换为 RGB
        if img.shape[-1] == 4:
            img = Image.fromarray(img).convert("RGB")
        else:
            img = Image.fromarray(img)

        # 保存为 RGB 格式的 PNG
        save_path = os.path.join(save_dir, f"sample_{i}.png")
        img.save(save_path, format="PNG")

    print(f"✅ 已成功保存 {num_images} 张 RGB 图片到 {save_dir}")





npz_path="/home/chunjie/improved-diffusion-main/ML_project/linear_schedule/samples_50000x32x32x3.npz"
save_dir="/home/chunjie/improved-diffusion-main/ML_project/linear_schedule/img"
save_npz_images(npz_path,save_dir,num_images=50000)





✅ 已成功保存 50000 张 RGB 图片到 /home/chunjie/improved-diffusion-main/ML_project/linear_schedule/img


In [2]:
# fid test
import torch_fidelity
# org ddim250
real_images_path = "/home/chunjie/improved-diffusion-main/datasets/cifar_train"
fake_images_path=save_dir
metrics = torch_fidelity.calculate_metrics(
    input1=real_images_path,
    input2=fake_images_path,
    cuda=True,
    isc=True, 
    fid=True, 
    kid=True,
    precision=True,  # 确保 Precision 计算
    recall=True,  # 确保 Recall 计算
)
print(metrics)  # 应该输出 FID, Precision, Recall

Creating feature extractor "inception-v3-compat" with features ['logits_unbiased', '2048']
Extracting features from input1
Looking for samples non-recursivelty in "/home/chunjie/improved-diffusion-main/datasets/cifar_train" with extensions png,jpg,jpeg
Found 50000 samples
/home/chunjie/miniconda3/envs/MyPytorch/lib/python3.13/site-packages/torch_fidelity/datasets.py:16: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(img.tobytes())).view(height, width, 3)
Processing samples                                                                                                                               
Extracting features from input2
Looking for samples non-recursivelty in "/home/chunjie/improved-diffus

{'inception_score_mean': 11.246055800480613, 'inception_score_std': 0.14352855727580247, 'frechet_inception_distance': 4.408335713021643, 'kernel_inception_distance_mean': 0.0031043338775634767, 'kernel_inception_distance_std': 0.0005775018845715509}


Kernel Inception Distance: 0.0031043338775634767 ± 0.0005775018845715509


In [2]:
import os

def count_images_in_path(directory):
    """
    统计指定路径及其子目录下的图片文件数量。

    Args:
        directory (str): 文件夹路径。

    Returns:
        int: 图片文件的总数量。
    """
    # 支持的图片文件扩展名
    valid_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".gif"}
    
    # 初始化图片计数
    image_count = 0

    # 遍历目录及其子目录
    for root, _, files in os.walk(directory):
        for file in files:
            # 检查文件扩展名是否为有效图片格式
            if os.path.splitext(file)[1].lower() in valid_extensions:
                image_count += 1
    
    return image_count

# 示例使用
directory_path = save_dir  # 替换为你的路径
total_images = count_images_in_path(directory_path)
print(f"路径 '{directory_path}' 下的图片数量为: {total_images}")

路径 '/home/chunjie/improved-diffusion-main/0420/train_data/' 下的图片数量为: 50000
